In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import graphics as g

In [ ]:
cell = "2d21b861"
epoch_idx = [5]
epochs = []
for e in epoch_idx:
    with open(f"../build/{cell}_{e:02}.pprox", "rt") as fp:
        epochs.append(json.load(fp))
epoch = epochs[0]

In [ ]:
epoch["pprox"][1]

In [ ]:
sweeps = (pd.json_normalize(epochs, "pprox", ["cell", "epoch", "timestamp", "bird", "sire", ["stats", "tau"], ["stats", "Cm"]])
          .rename(columns={"index": "sweep", "stats.tau": "tau", "stats.Cm": "Cm"})
          .set_index(["cell", "epoch", "sweep"])
          .drop(columns=["offset", "interval"])
         )
sweeps.iloc[-1]

In [ ]:
sweeps.iloc[-1].events

In [ ]:
cells = sweeps.reset_index()[["cell","bird", "sire"]].drop_duplicates().set_index("cell")

In [ ]:
epochs = (
        sweeps.reset_index()[["cell","epoch","timestamp","tau","Cm"]]
        .drop_duplicates()
        .set_index(["cell", "epoch"])
)
ts = epochs.pop("timestamp").apply(pd.Timestamp)
epochs["time"] = ts.groupby(["cell"], group_keys=False).apply(lambda x: (x - x.iloc[0])).apply(lambda x: x.total_seconds())

In [ ]:
sweep = sweeps.iloc[0]
def sweep_iv_stats(sweep):
    nsteps = len(sweep["steps.I"])
    return pd.Series(
        np.concatenate([sweep["steps.I"], sweep["steps.V"]]),
        index=pd.MultiIndex.from_product([["current", "voltage"], range(nsteps)], names=["value", "step"])
    )#.swaplevel().sort_index()
iv_stats = sweeps.apply(sweep_iv_stats, axis=1)
dev = (iv_stats.voltage - iv_stats.voltage.median()).abs()
v_dev = dev / dev.median()
bad_sweeps = (v_dev[[0,2,3,4]] > 10).any(axis=1)
iv_stats = iv_stats.loc[~bad_sweeps].stack("step")

In [ ]:
iv_stats

In [ ]:
v_dev

In [ ]:
plt.plot(iv_stats.current, iv_stats.voltage, "ro")

In [ ]:
from loess.loess_1d import loess_1d
I_min = iv_stats.current.min()
I_max = iv_stats.current.max()
bins = pd.interval_range(np.floor(I_min), np.ceil(I_max))
cuts = pd.cut(iv_stats.current, bins)
binned = iv_stats.groupby(cuts).agg('mean').dropna()
xnew = np.arange(0, 60, 0.1)
#xnew = np.arange(I_min, I_max, 1.0)
xout, yout, wout = loess_1d(binned.current.to_numpy(), binned.voltage.to_numpy(), xnew=xnew, degree=1, frac=0.7, npoints=None, rotate=False, sigy=None)
smoothed = pd.DataFrame({"current": xout, "voltage": yout})
plt.plot(smoothed.current, smoothed.voltage)
plt.plot(binned.current, binned.voltage, "ro")
(smoothed.voltage.diff() / smoothed.current.diff()).mean()

In [ ]:
iv_stats.index[0][0]

In [ ]:
def sweep_firing_stats(sweep):
    try:
        step = pd.Interval(*sweep["stimulus.interval"])
        spikes = [e for e in sweep.events if e in step]
        rate = len(spikes) / step.length
        if len(spikes) == 0:
            duration = np.nan
        elif len(spikes) == 1:
            duration = (
                sweep["first_spike.width"] + sweep["first_spike.trough_t"]
            ) / 1000.0
        else:
            duration = spikes[-1] - spikes[0]
    except TypeError:
        rate = duration = np.nan
    return pd.Series(
        {
            "current": sweep["stimulus.I"],
            "firing_rate": rate,
            "firing_duration": duration,
            "Rs": sweep.Rs,
            "Rm": sweep.Rm,
            "Vm": sweep.Vm,
            "temperature": sweep.temperature,
        }
    )

sweep_stats = sweeps.apply(sweep_firing_stats, axis=1)

In [ ]:
sweep_stats

In [ ]:
(idx,) = (sweep_stats.firing_rate > 0).to_numpy().nonzero()
slope = sweep_stats.firing_rate.diff() / sweep_stats.current.diff()
slope[:idx[0]] = np.nan
x = slope.first_valid_index()
slope.index.get_loc(None)

In [ ]:
def epoch_stats(sweeps):
    # sweeps with spikes
    idx, = (sweep_stats.firing_rate > 0).to_numpy().nonzero()
    df = sweep_stats.iloc[idx[0]-1:]
    I_0 = (df.current[1] + df.current[0]) / 2
    slope = np.mean(np.diff(df.firing_rate) / np.diff(df.current))
    return pd.Series([
    sweeps.firing_duration.max(),
    sweeps.firing_duration.mean(),
    sweeps.firing_rate.max(),
        I_0,
        slope,
    sweeps.Rs.mean(),
    sweeps.Rm.mean(),
    sweeps.temperature.mean(),
    ], index=["duration_max", "duration_mean", "rate_max", "rheobase", "fI_slope", "Rs", "Rm", "T"])
epoch_stats(sweep_stats)

In [ ]:
estats = pd.read_csv("../build/epoch_stats.csv", index_col=["cell", "epoch"])
estats

In [ ]:
df = estats.loc[cell]
(df["n_spont"] + df["n_evoked"]).shift(1, fill_value=0).cumsum()

In [ ]:
x = estats.groupby("cell", group_keys=False).apply(lambda df: (df["n_spont"] + df["n_evoked"]).shift(1, fill_value=0).cumsum()).rename("cum_spikes")

In [ ]:
x

In [ ]:
estats.join([x])